# CircleCI Compute Credits Analysis Report

## Setup and Configuration

In [ ]:
# Setup and configuration
import analysis
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Initialize the analysis environment
helpers = analysis.initialize_notebook()
pp = helpers['pp']
summarize_dataset = helpers['summarize_dataset']

# Configuration - can be set via environment variables or modified directly
filepath = os.getenv("FILEPATH", "../merged.csv")
project_name = os.getenv("PROJECT_NAME", "project_10") 
individual_job_name = os.getenv("JOB_NAME", "deploy")
credit_cost = float(os.getenv("CREDIT_COST", "0.0006"))
top_n_projects = 15  # Number of top projects to display in charts

# Load data to get organization name for report header
temp_df = pd.read_csv(filepath, escapechar="\\", na_values=["\\N"], nrows=1)
org_name = temp_df['ORGANIZATION_NAME'].iloc[0] if 'ORGANIZATION_NAME' in temp_df.columns else "Unknown Organization"

print("=" * 60)
print("CircleCI Compute Credits Analysis Report")
print("=" * 60)
print(f"Organization: {org_name}")
print(f"Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print()
print(f"Configuration:")
print(f"  File: {filepath}")
print(f"  Credit Cost: {credit_cost}")
print(f"  Top N Projects: {top_n_projects}")


In [ ]:
# Set matplotlib backend for headless environments
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)


In [ ]:
# Load the CircleCI usage data
print(f"Loading data from: {filepath}")
try:
    df = pd.read_csv(filepath, escapechar="\\", na_values=["\\N"])
    print(f"✅ Successfully loaded {len(df):,} rows of data")
    print(f"📊 Date range: {df['PIPELINE_CREATED_AT'].min()} to {df['PIPELINE_CREATED_AT'].max()}")
except FileNotFoundError:
    print(f"❌ Error: Could not find data file at {filepath}")
    print("Please ensure the merged.csv file exists in the expected location.")
    raise
except Exception as e:
    print(f"❌ Error loading data: {str(e)}")
    raise



In [ ]:
# Data preparation and validation
print("🔍 Data validation and preparation...")

# Check for required columns
required_columns = ['PROJECT_NAME', 'COMPUTE_CREDITS', 'JOB_RUN_NUMBER']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"❌ Missing required columns: {missing_columns}")
    print(f"Available columns: {list(df.columns)}")
    raise ValueError(f"Missing required columns: {missing_columns}")

# Filter to valid jobs with compute credits data
initial_count = len(df)
df = df[df['JOB_RUN_NUMBER'].notna()]
df = df[df['COMPUTE_CREDITS'].notna()]
df = df[df['PROJECT_NAME'].notna()]

print(f"📈 Filtered from {initial_count:,} to {len(df):,} valid job records")
print(f"📊 Found {df['PROJECT_NAME'].nunique()} unique projects")
print(f"💰 Total compute credits in dataset: {df['COMPUTE_CREDITS'].sum():,.0f}")



## Compute Credits Analysis by Project


In [ ]:
# Aggregate compute credits by project
project_stats = df.groupby('PROJECT_NAME').agg({
    'COMPUTE_CREDITS': ['sum', 'mean', 'count'],
    'JOB_RUN_NUMBER': 'count'
}).round(2)

# Flatten column names
project_stats.columns = ['TOTAL_COMPUTE_CREDITS', 'AVG_CREDITS_PER_JOB', 'CREDIT_RECORDS', 'TOTAL_JOBS']
project_stats = project_stats.reset_index()

# Add cost calculations
project_stats['TOTAL_COST'] = project_stats['TOTAL_COMPUTE_CREDITS'] * credit_cost
project_stats['AVG_COST_PER_JOB'] = project_stats['AVG_CREDITS_PER_JOB'] * credit_cost

# Sort by total compute credits
project_stats = project_stats.sort_values('TOTAL_COMPUTE_CREDITS', ascending=False)

print(f"📊 Analysis complete for {len(project_stats)} projects")
print(f"💰 Total compute credits across all projects: {project_stats['TOTAL_COMPUTE_CREDITS'].sum():,.0f}")
print(f"💵 Total estimated cost: ${project_stats['TOTAL_COST'].sum():,.2f}")



## Top Projects by Compute Credits


In [ ]:
# Display top projects table
top_projects = project_stats.head(top_n_projects)

# Format the table for display
display_table = top_projects[['PROJECT_NAME', 'TOTAL_COMPUTE_CREDITS', 'TOTAL_JOBS', 'AVG_CREDITS_PER_JOB', 'TOTAL_COST']].copy()
display_table['TOTAL_COMPUTE_CREDITS'] = display_table['TOTAL_COMPUTE_CREDITS'].apply(lambda x: f"{x:,.0f}")
display_table['TOTAL_JOBS'] = display_table['TOTAL_JOBS'].apply(lambda x: f"{x:,.0f}")
display_table['AVG_CREDITS_PER_JOB'] = display_table['AVG_CREDITS_PER_JOB'].apply(lambda x: f"{x:.1f}")
display_table['TOTAL_COST'] = display_table['TOTAL_COST'].apply(lambda x: f"${x:,.2f}")

display_table.columns = ['Project Name', 'Total Compute Credits', 'Total Jobs', 'Avg Credits/Job', 'Total Cost']

print(f"🏆 Top {top_n_projects} Projects by Compute Credits Usage")
print("=" * 80)
display(HTML(display_table.to_html(index=False, escape=False, classes='table table-striped')))



## Visual Analysis


In [ ]:
# Chart 1: Total Compute Credits by Project (Top N)
plt.figure(figsize=(14, 8))
top_projects_plot = project_stats.head(top_n_projects)

bars = plt.bar(range(len(top_projects_plot)), top_projects_plot['TOTAL_COMPUTE_CREDITS'], 
               color='steelblue', alpha=0.7, edgecolor='black', linewidth=0.5)

plt.title(f'Total Compute Credits by Project (Top {top_n_projects})', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Project', fontsize=12)
plt.ylabel('Total Compute Credits', fontsize=12)
plt.xticks(range(len(top_projects_plot)), top_projects_plot['PROJECT_NAME'], rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{height:,.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Summary statistics
total_credits = project_stats['TOTAL_COMPUTE_CREDITS'].sum()
top_5_credits = project_stats.head(5)['TOTAL_COMPUTE_CREDITS'].sum()
print(f"📊 Top 5 projects account for {(top_5_credits/total_credits*100):.1f}% of all compute credits")



In [ ]:
# Chart 2: Distribution of Compute Credits
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histogram of compute credits distribution
ax1.hist(project_stats['TOTAL_COMPUTE_CREDITS'], bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
ax1.set_title('Distribution of Total Compute Credits\nAcross All Projects', fontweight='bold')
ax1.set_xlabel('Total Compute Credits')
ax1.set_ylabel('Number of Projects')
ax1.grid(True, alpha=0.3)

# Pie chart of top projects
top_8_projects = project_stats.head(8)
other_credits = project_stats.iloc[8:]['TOTAL_COMPUTE_CREDITS'].sum()

pie_data = list(top_8_projects['TOTAL_COMPUTE_CREDITS']) + [other_credits]
pie_labels = list(top_8_projects['PROJECT_NAME']) + ['Others']

ax2.pie(pie_data, labels=pie_labels, autopct='%1.1f%%', startangle=90)
ax2.set_title('Compute Credits Distribution\n(Top 8 Projects + Others)', fontweight='bold')

plt.tight_layout()
plt.show()



In [ ]:
# Chart 3: Average Credits per Job Analysis
plt.figure(figsize=(14, 8))

# Filter to projects with significant job volume for meaningful averages
significant_projects = project_stats[project_stats['TOTAL_JOBS'] >= 100].head(top_n_projects)

if len(significant_projects) > 0:
    bars = plt.bar(range(len(significant_projects)), significant_projects['AVG_CREDITS_PER_JOB'], 
                   color='lightgreen', alpha=0.7, edgecolor='black', linewidth=0.5)
    
    plt.title(f'Average Compute Credits per Job\n(Projects with 100+ jobs)', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Project', fontsize=12)
    plt.ylabel('Average Compute Credits per Job', fontsize=12)
    plt.xticks(range(len(significant_projects)), significant_projects['PROJECT_NAME'], rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                 f'{height:.1f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No projects found with 100+ jobs for meaningful average analysis")



## Summary Statistics


In [ ]:
# Generate summary statistics
print("📊 COMPUTE CREDITS ANALYSIS SUMMARY")
print("=" * 50)
print(f"Total Projects Analyzed: {len(project_stats)}")
print(f"Total Compute Credits: {project_stats['TOTAL_COMPUTE_CREDITS'].sum():,.0f}")
print(f"Total Estimated Cost: ${project_stats['TOTAL_COST'].sum():,.2f}")
print(f"Average Credits per Project: {project_stats['TOTAL_COMPUTE_CREDITS'].mean():.0f}")
print(f"Median Credits per Project: {project_stats['TOTAL_COMPUTE_CREDITS'].median():.0f}")
print()
print("🏆 TOP CONSUMER INSIGHTS:")
print(f"Highest consuming project: {project_stats.iloc[0]['PROJECT_NAME']}")
print(f"  - Total credits: {project_stats.iloc[0]['TOTAL_COMPUTE_CREDITS']:,.0f}")
print(f"  - Total cost: ${project_stats.iloc[0]['TOTAL_COST']:,.2f}")
print(f"  - Jobs run: {project_stats.iloc[0]['TOTAL_JOBS']:,.0f}")
print()
print("💡 DISTRIBUTION INSIGHTS:")
top_5_percentage = (project_stats.head(5)['TOTAL_COMPUTE_CREDITS'].sum() / project_stats['TOTAL_COMPUTE_CREDITS'].sum()) * 100
top_10_percentage = (project_stats.head(10)['TOTAL_COMPUTE_CREDITS'].sum() / project_stats['TOTAL_COMPUTE_CREDITS'].sum()) * 100
print(f"Top 5 projects account for: {top_5_percentage:.1f}% of total credits")
print(f"Top 10 projects account for: {top_10_percentage:.1f}% of total credits")

# Cost analysis
if project_stats['TOTAL_COST'].sum() > 1000:
    print(f"\n💰 COST OPTIMIZATION OPPORTUNITY:")
    print(f"Monthly cost projection: ${project_stats['TOTAL_COST'].sum() * 30:.2f}")
    print(f"Focus optimization efforts on top {min(5, len(project_stats))} projects for maximum impact")

